In [ ]:

import matplotlib as plt
import numpy as np
import pandas as pd
import langid
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy import sparse
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_all_tweet = pd.read_csv('Latest_Tweet_Fake_Spreader_Tweets.csv')

In [ ]:
df_acc_manual_pol = pd.read_csv('Political Orientation Manual 1.csv', dtype = 'str')

In [ ]:
df_acc_manual_pol = df_acc_manual_pol.loc[df_acc_manual_pol.User_ID != '7.2296E+17']

In [ ]:
df_acc_manual_pol = df_acc_manual_pol.dropna()
df_acc_manual_pol.User_ID= df_acc_manual_pol.User_ID.astype('int')

In [ ]:
df_acc_manual_pol.User_ID

In [ ]:
df_pol = df_account.loc[df_account.Political == 'Political']

In [ ]:
df_account[df_account.User_ID.isin(df_acc_manual_pol.User_ID)]

In [ ]:
df_pol2= df_account[df_account.User_ID.isin(df_acc_manual_pol.User_ID)]

In [ ]:
df_pol2.columns

In [ ]:
df_pol = pd.concat([df_pol,df_pol2])[['User_ID', 'UserDescription', 'Political', 'party']]

In [ ]:
df_acc_manual_pol.columns 

In [ ]:
df_pol.columns

In [ ]:
merged_df = pd.merge(df_pol, df_acc_manual_pol, on='User_ID', suffixes=('_pol', '_manual_pol'), how='left')

merged_df['Political'] = merged_df['Political_manual_pol'].fillna(merged_df['Political_pol'])
merged_df['Party'] = merged_df['Party'].fillna(merged_df['party'])



In [ ]:
df_pol_final = merged_df[['User_ID', 'UserDescription', 'Political', 'Party']]

In [ ]:
df_pol_final.groupby('Political').count()

In [ ]:
df_pol_final.loc[df_pol_final.Political == 'Non political', 'Political'] = 'Non Political'

In [ ]:
df_pol_final.loc[df_pol_final.Political == 'Non political']

In [ ]:
df_pol_final = df_pol_final.drop('political', axis = 1)

In [ ]:
df_pol_final.groupby('Political').count()

In [ ]:
df_pol_final.groupby('Party').count()

In [ ]:
df_acc_manual_pol.groupby('Party').count()

In [ ]:
df_account.groupby('party').count()

In [ ]:
df_pol_final2 = df_pol_final.dropna()

In [ ]:
df_pol_final2.groupby('Party').count()

In [ ]:
df_pol_final3 = df_pol_final2.loc[df_pol_final2.Party.isin(['AAP', 'BJP', 'INC', 'No'])]

In [ ]:
pip install langid

In [ ]:
df_pol_final3.columns

In [ ]:

def detect_language(text):
    return langid.classify(text)[0]

df_pol_final3 ['language'] = df_pol_final3 ['UserDescription'].apply(detect_language)

tfidf_vectorizer_en = TfidfVectorizer(max_features=300, stop_words='english')
tfidf_vectorizer_hi = TfidfVectorizer(max_features=300)  


In [ ]:
df_pol_final3.loc[df_pol_final3.language!= 'en', 'language'] = 'hi'

In [ ]:
X_en = tfidf_vectorizer_en.fit_transform(df_pol_final3[df_pol_final3['language'] == 'en']['UserDescription'] )
X_hi = tfidf_vectorizer_hi.fit_transform(df_pol_final3[df_pol_final3['language'] == 'hi']['UserDescription'] )

X = sparse.vstack([X_en, X_hi])
y = df_pol_final3['Party']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)



In [ ]:
X

In [ ]:
classifier = classifier = LogisticRegression(multi_class='auto', solver='lbfgs')
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Multinomial Naive Bayes": MultinomialNB()
}

for name, classifier in classifiers.items():
    print(f"Training {name}...")
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of {name}: {accuracy:.2f}")
    print(f"Classification Report of {name}:\n{classification_report(y_test, y_pred)}")

In [ ]:
df_pol_final3.groupby('Party').count()

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

classifier = RandomForestClassifier() 
classifier.fit(X_train_resampled, y_train_resampled)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))